In [1]:
using SDDP, Distributions, GLPK, Plots

In [2]:
graph = SDDP.Graph(
           :root_node,
           [:decision_node,:demand_node],
           [
               (:root_node => :decision_node, 1.0),
               (:decision_node => :demand_node, 1.0)
           ]);


p = 1.0 #day ahead price
q = 2.0 #same day price

model = SDDP.PolicyGraph(
            graph,
            sense = :Min,
            lower_bound = 0,
            optimizer = with_optimizer(GLPK.Optimizer)) do subproblem, node

                @variable(subproblem, stock>=0, SDDP.State, initial_value = 0)

                @variable(subproblem, 0 <= reserve)
                @variable(subproblem, 0 <= shortage)

                @variable(subproblem, demand)

                @constraint(subproblem, stock.out == stock.in + reserve + shortage - demand)

                if node == :decision_node
                        JuMP.fix(demand,0.0)
                        JuMP.fix(shortage,0.0;force=true)
                else

                    ##demandas uniformes
                    demand_range = collect(0:100);
                    demand_probability = ones(size(demand_range));
                    demand_probability ./= sum(demand_probability);

                    SDDP.parameterize(subproblem,demand_range,demand_probability) do d
                        JuMP.fix(demand,d)
                        JuMP.fix(reserve,0.0;force=true)
                    end
                end


                if node == :decision_node
                    @stageobjective(subproblem,  p*reserve);
                else
                    @stageobjective(subproblem,  q*shortage);
                end
            end

A policy graph with 2 nodes.
 Node indices: decision_node, demand_node


In [3]:
SDDP.train(model,iteration_limit=20)
#AVAR version
#SDDP.train(model,risk_measure = SDDP.AVaR(1.0),iteration_limit=20)

--------------------------------------------------------------------------------
                      SDDP.jl (c) Oscar Dowson, 2017-20

Numerical stability report
  Non-zero Matrix range     [1e+00, 1e+00]
  Non-zero Objective range  [1e+00, 2e+00]
  Non-zero Bounds range     [0e+00, 0e+00]
  Non-zero RHS range        [0e+00, 0e+00]
No problems detected

Solver: serial mode

 Iteration    Simulation       Bound         Time (s)    Proc. ID   # Solves
        1    1.460000e+02   5.000000e+01   1.330057e+01          1        104
        2    5.800000e+01   7.500000e+01   1.335031e+01          1        208
        3    1.450000e+02   7.512871e+01   1.335181e+01          1        312
        4    3.800000e+01   7.519307e+01   1.335336e+01          1        416
        5    1.175000e+02   7.522277e+01   1.335500e+01          1        520
        6    7.450000e+01   7.523762e+01   1.335672e+01          1        624
        7    1.070000e+02   7.524752e+01   1.335843e+01          1        7

In [4]:
results=SDDP.simulate(model,1,[:reserve,:stock,:demand,:shortage])

1-element Array{Array{Dict{Symbol,Any},1},1}:
 [Dict(:stock => SDDP.State{Float64}(0.0, 49.99999999999938),:bellman_term => 25.24752475247586,:noise_term => nothing,:node_index => :decision_node,:stage_objective => 49.99999999999938,:objective_state => nothing,:reserve => 49.99999999999938,:demand => 0.0,:belief => Dict(:decision_node => 1.0),:shortage => 0.0…), Dict(:stock => SDDP.State{Float64}(49.99999999999938, 0.0),:bellman_term => 0.0,:noise_term => 94,:node_index => :demand_node,:stage_objective => 88.00000000000124,:objective_state => nothing,:reserve => 0.0,:demand => 94.0,:belief => Dict(:demand_node => 1.0),:shortage => 44.00000000000062…)]